# Temperature Prediction Experiments with Multiple Models

This notebook compares R2C2, Ridge, and Random Forest models for indoor temperature prediction across different noise levels.

## Imports and Configuration

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from collections import OrderedDict

from scipy.linalg import expm, inv
from scipy.stats import norm
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

## Load R2C2 Optimization Results

In [ ]:
def read_optimization_results_from_csv(sensor_counts):
    """
    Reads R2C2 optimization results from CSV files.

    Parameters:
    - sensor_counts: List of sensor counts to identify which CSV files to read

    Returns:
    - Dictionary with structure [sensor_count][house_id] containing optimization outcomes
    """
    optimization_results = {}
    for sensor_count in sensor_counts:
        file_name = f"/Users/ozanbaris/Documents/GitHub/TS-foundation-model/R2C2_models/decent_R2C2_optimization_results_sensor_{sensor_count}.csv"
        if os.path.exists(file_name):
            df = pd.read_csv(file_name)
            houses = {}
            for _, row in df.iterrows():
                house_id = row['House ID']
                results = {
                    'rmse_train': row['RMSE Train'],
                    'rmse_test': row['RMSE Test'],
                    'optimal_params': {
                        'Ri': list(map(float, row['Ri_values'].split(', '))),
                        'Re': list(map(float, row['Re_values'].split(', '))),
                        'Ci': list(map(float, row['Ci_values'].split(', '))),
                        'Ce': list(map(float, row['Ce_values'].split(', '))),
                        'Ai': list(map(float, row['Ai_values'].split(', '))),
                        'Ae': list(map(float, row['Ae_values'].split(', '))),
                        'roxP_hvac': list(map(float, row['roxP_hvac'].split(', ')))
                    }
                }
                houses[house_id] = results
            
            optimization_results[sensor_count] = houses
        else:
            print(f"File '{file_name}' not found.")
    
    return optimization_results


sensor_counts = [1, 2, 3, 4, 5] 
optimization_results_R2C2_decent = read_optimization_results_from_csv(sensor_counts)

## Model Definitions

In [ ]:
class R2C2Model:
    """
    R2C2 (2 Resistances, 2 Capacitances) thermal model for building temperature prediction.
    """
    def __init__(self, Ri, Re, Ci, Ce, Ai, Ae, roxP_hvac):
        self.Ri = Ri
        self.Re = Re
        self.Ci = Ci
        self.Ce = Ce
        self.Ai = Ai
        self.Ae = Ae
        self.roxP_hvac = roxP_hvac
        self.N_states = 2
        self.update_matrices()

    def update_matrices(self):
        """Update continuous-time state-space matrices."""
        self.Ac = np.array([[-1/(self.Ci*self.Ri), 1/(self.Ci*self.Ri)],
                            [1/(self.Ce*self.Ri), -1/(self.Ce*self.Ri) - 1/(self.Ce*self.Re)]])
        self.Bc = np.array([[0, -self.roxP_hvac / self.Ci, self.Ai / self.Ci],
                            [1/(self.Ce*self.Re), 0, self.Ae/self.Ce]])
        self.Cc = np.array([[1, 0]])

    def discretize(self, dt):
        """Discretize continuous-time model."""
        n = self.N_states
        F = expm(self.Ac * dt)
        G = np.dot(inv(self.Ac), np.dot(F - np.eye(n), self.Bc))
        H = self.Cc
        return F, G
    
    def predict_onestep(self, T, Te, T_ext, u, ghi, dt):
        """
        One-step temperature prediction using vectorized operations.
        
        Parameters:
        - T: Indoor temperatures (K)
        - Te: Envelope temperatures (K)
        - T_ext: External temperatures (K)
        - u: HVAC duty cycle (fraction of time on)
        - ghi: Solar radiation (W/m²)
        - dt: Time step (seconds)
        
        Returns:
        - Predicted indoor temperatures (K)
        """
        F, G = self.discretize(dt)
        
        # Stack T and Te vertically, then transpose to get a 2xN matrix where N is the number of samples
        state_matrix = np.vstack((T, Te)).T  # Each row is [T, Te] for a timestep
        
        # Similarly, stack T_ext, u, and ghi vertically and transpose
        input_matrix = np.vstack((T_ext, u, ghi)).T  # Each row is [T_ext, u, ghi] for a timestep
        
        # Perform the matrix multiplication in a batch
        predictions = (F @ state_matrix.T) + (G @ input_matrix.T)
        
        # Transpose the result back so each row corresponds to a timestep
        predictions = predictions.T
        
        # And only return the first column, which represents T
        return predictions[:, 0]
    
    def predict_two(self, T, Te, T_ext, u, ghi, dt):
        """Predict both indoor and envelope temperatures."""
        F, G = self.discretize(dt)
        state_matrix = np.vstack((T, Te)).T
        input_matrix = np.vstack((T_ext, u, ghi)).T
        predictions = (F @ state_matrix.T) + (G @ input_matrix.T)
        predictions = predictions.T
        return predictions[:, 0], predictions[:, 1]
    
    def autoregressive_predict(self, T_init, Te_init, T_ext_seq, u_seq, ghi_seq, dt, pred_hrz=64):
        """
        Autoregressive temperature predictions.

        Parameters:
        - T_init: Initial indoor temperature (K)
        - Te_init: Initial envelope temperature (K)
        - T_ext_seq: Sequence of external temperatures (K)
        - u_seq: Sequence of HVAC duty cycles
        - ghi_seq: Sequence of solar radiation values (W/m²)
        - dt: Time step (seconds)
        - pred_hrz: Prediction horizon (number of steps)

        Returns:
        - Array of predicted indoor temperatures (°F)
        """
        T = np.zeros(pred_hrz)
        Te = Te_init
        preds = []
        
        for t in range(0, pred_hrz):
            if t == 0:
                T_pred = self.predict_onestep(
                    T_init, Te_init,
                    T_ext_seq[0], u_seq[0], ghi_seq[0], dt
                )
            else:
                T_pred, _ = self.predict_two(
                    T_prev, Te,
                    T_ext_seq[t], u_seq[t], ghi_seq[t], dt
                )
            preds.append(T_pred)
            T_prev = T_pred
            Te = (T_pred + T_ext_seq[t]) / 2

        # Convert Kelvin to Fahrenheit
        preds = (np.array(preds) - 273.15) * 9/5 + 32
        
        return preds

## Data Loading and Processing Utilities

In [ ]:
def read_csvs_to_dfs(main_output_directory):
    """
    Reads all house CSV files from subdirectories.
    
    Parameters:
    - main_output_directory: Root directory containing subdirectories with house data
    
    Returns:
    - Dictionary with structure [house_group][house_id] containing DataFrames
    """
    all_houses_dict = {}
    
    for subdirectory in os.listdir(main_output_directory):
        sub_output_directory = os.path.join(main_output_directory, subdirectory)
        
        if not os.path.isdir(sub_output_directory):
            continue
        
        house_group = int(subdirectory.split("_")[-1])
        
        if house_group not in all_houses_dict:
            all_houses_dict[house_group] = {}
        
        for filename in os.listdir(sub_output_directory):
            if filename.endswith(".csv"):
                file_path = os.path.join(sub_output_directory, filename)
                house_id = filename.split("_")[-1].replace(".csv", "")
                df = pd.read_csv(file_path)
                all_houses_dict[house_group][house_id] = df
                
    return all_houses_dict


def process_house_data(df):
    """
    Processes a single house DataFrame: normalizes duty cycle, renames columns, 
    converts temperatures to Kelvin, and handles missing values.
    """
    df['duty_cycle'] = df['CoolingRunTime'] / 3600
    df.rename(columns={'Outdoor_Temperature': 'Text'}, inplace=True)
    
    sensor_rename_map = {
        'Thermostat_Temperature': 'T01_TEMP',
        'RemoteSensor1_Temperature': 'T02_TEMP',
        'RemoteSensor2_Temperature': 'T03_TEMP',
        'RemoteSensor3_Temperature': 'T04_TEMP',
        'RemoteSensor4_Temperature': 'T05_TEMP',
        'RemoteSensor5_Temperature': 'T06_TEMP',
    }
    df.rename(columns=sensor_rename_map, inplace=True)

    temp_columns = [f"T0{i}_TEMP" for i in range(1, 7)] + ['Text']
    for col in temp_columns:
        df[col] = (df[col] - 32) * 5/9 + 273.15

    columns_to_keep = ['time', 'GHI', 'duty_cycle'] + temp_columns
    df = df[columns_to_keep]
    df.fillna(method='ffill', inplace=True)

    return df


def prepare_experiment_data(dataset_name, optimization_results):
    """
    Loads and prepares all experiment data including house rankings and flattened results.
    
    Parameters:
    - dataset_name: Name of the dataset directory
    - optimization_results: R2C2 optimization results dictionary
    
    Returns:
    - Tuple of (new_house_data, flattened_results, flattened_data)
    """
    main_output_directory = f"/Users/ozanbaris/Documents/GitHub/TS-foundation-model/{dataset_name}"
    all_houses_reduced = read_csvs_to_dfs(main_output_directory)
    
    # Flatten and sort the house data
    new_house_data = {}
    total = 0
    house_data = OrderedDict(sorted(all_houses_reduced.items()))
    
    for house_group in house_data:
        sorted_dict = OrderedDict(sorted(house_data[house_group].items()))
        for house_id in sorted_dict:
            total += 1
            new_house_data[house_id] = total
    
    # Process houses
    processed_houses_reduced = {}
    for sensor_count, houses in all_houses_reduced.items():
        processed_houses = {}
        for house_id, df in houses.items():
            if 'GHI' not in df.columns:
                print(f"Skipping house {house_id} due to missing 'GHI' column.")
                continue
            processed_houses[house_id] = process_house_data(df.copy())
        processed_houses_reduced[sensor_count] = processed_houses
    
    # Flatten optimization results
    flattened_results = {}
    for sensor_count, houses in optimization_results.items():
        for house_id, result in houses.items():
            flattened_results[house_id] = result
    
    # Flatten processed data
    flattened_data = {}
    for sensor_count, houses in processed_houses_reduced.items():
        for house_id, df in houses.items():
            flattened_data[house_id] = df
    
    return new_house_data, flattened_results, flattened_data

## Metrics and Results Utilities

In [ ]:
def calc_normalized_rmse(y_true, y_pred):
    """
    Calculate normalized RMSE.
    
    Returns:
    - RMSE normalized by the range of ground truth values
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    range_of_data = np.max(y_true) - np.min(y_true)
    normalized_rmse = rmse / (range_of_data + 1e-20)
    
    return normalized_rmse


def calculate_metrics(gt, forecast):
    """
    Calculate MSE, RMSE, and normalized RMSE.
    
    Returns:
    - Tuple of (mse, rmse, normalized_rmse)
    """
    gt = np.float64(gt)
    if np.sum(np.isnan(forecast)) >= 1 or np.sum(np.isnan(gt)) >= 1:
        print('NaN detected...')
    
    mse = np.mean((gt - forecast) ** 2)
    rmse = np.sqrt(mse)
    normalized_rmse = calc_normalized_rmse(gt, forecast)
    return mse, rmse, normalized_rmse


def save_results_for_model(model_name, ground_truth, timestamp_forecast, forecast, 
                           duration, pred_hrz, mode, occupancy, batch_id, directory):
    """
    Save model predictions and metrics to CSV file.
    """
    mse, rmse, normalized_rmse = calculate_metrics(ground_truth, forecast)

    result = {
        'batch_id': batch_id,
        'MSE': mse,
        'RMSE': rmse, 
        'Norm_RMSE': normalized_rmse, 
        'GroundTruth': ground_truth.tolist(),
        'Timestamp_forecast': timestamp_forecast.tolist(),
        'Forecast': forecast.tolist(),
        'Duration': duration,
        'PredictionHorizon': pred_hrz,
        'Mode': mode,
        'Occupancy': occupancy,
        'Model': model_name
    }

    file_path = f"{directory}/results_{mode}_{occupancy}/{duration}_{pred_hrz}/{model_name}.csv"
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        df = pd.DataFrame(columns=['batch_id', 'MSE', 'RMSE', 'Norm_RMSE', 'Mode', 
                                   'Occupancy', 'Duration', 'PredictionHorizon', 'Data', 
                                   'Timestamp_data', 'GroundTruth', 'Timestamp_forecast', 
                                   'Forecast', 'Model'])

    df = pd.concat([df, pd.DataFrame([result])], ignore_index=True)
    df.to_csv(file_path, index=False)


def compute_metrics(ground_truth, forecast):
    """
    Compute RMSE metrics for different prediction horizons.

    Returns:
    - Dictionary with RMSE for all predictions, 1st, 6th, and 24th steps
    """
    metrics = {
        'RMSE_all': np.sqrt(np.mean((ground_truth - forecast) ** 2)),
        'RMSE_1st': np.sqrt(np.mean((ground_truth[:, 0] - forecast[:, 0]) ** 2)) if forecast.shape[1] > 0 else None,
        'RMSE_6th': np.sqrt(np.mean((ground_truth[:, 5] - forecast[:, 5]) ** 2)) if forecast.shape[1] > 5 else None,
        'RMSE_24th': np.sqrt(np.mean((ground_truth[:, 23] - forecast[:, 23]) ** 2)) if forecast.shape[1] > 23 else None
    }
    return metrics


def extract_valid_occupancies(flattened_data, flattened_results, new_house_data):
    """
    Extract valid occupancy ranks for houses with complete data.
    """
    valid_occupancies = []
    for house_id, house_data in flattened_data.items():
        if house_id in flattened_results:
            rank = new_house_data[house_id]
            valid_occupancies.append(rank)
        else:
            print(f"Skipping house ID: {house_id} due to missing optimization results.")
    return valid_occupancies

## Autoregressive Prediction Functions

In [ ]:
def autoregressive_predict_ridge(ridge_model, test_df, pred_hrz=24):
    """
    Autoregressive forecast for Ridge regression with covariates.
    Predicts T(t+1) from [Text(t), duty_cycle(t), GHI(t), T01_TEMP(t)].
    
    Returns:
    - Temperature predictions in Fahrenheit
    """
    preds_K = []
    T_init = test_df['T01_TEMP'].iloc[0]
    T_prev = T_init
  
    for t in range(0, pred_hrz):
        features = {
            'Text': test_df['Text'].iloc[t],
            'duty_cycle': test_df['duty_cycle'].iloc[t],
            'GHI': test_df['GHI'].iloc[t],
            'T01_TEMP': T_prev
        }
        X = pd.DataFrame([features])
        T_next = ridge_model.predict(X)[0]
        preds_K.append(T_next)
        T_prev = T_next

    T_pred_F = (np.array(preds_K) - 273.15) * 9/5 + 32
    return T_pred_F


def autoregressive_predict_rf(rf_model, test_df, pred_hrz=24):
    """
    Autoregressive forecast for Random Forest with covariates.
    Predicts T(t+1) from [Text(t), duty_cycle(t), GHI(t), T01_TEMP(t)].
    
    Returns:
    - Temperature predictions in Fahrenheit
    """
    preds_K = []
    T_init = test_df['T01_TEMP'].iloc[0]
    T_prev = T_init
    
    for t in range(0, pred_hrz):
        features = {
            'Text': test_df['Text'].iloc[t],
            'duty_cycle': test_df['duty_cycle'].iloc[t],
            'GHI': test_df['GHI'].iloc[t],
            'T01_TEMP': T_prev
        }
        X = pd.DataFrame([features])
        T_next = rf_model.predict(X)[0]
        preds_K.append(T_next)
        T_prev = T_next

    T_pred_F = (np.array(preds_K) - 273.15) * 9/5 + 32
    return T_pred_F


def autoregressive_predict_ridge_univariate(ridge_model, test_df, pred_hrz=24):
    """
    Univariate autoregressive forecast for Ridge.
    Predicts T(t+1) using only T(t).
    
    Note: Model must have been trained using only 'T01_TEMP' as feature.
    
    Returns:
    - Temperature predictions in Fahrenheit
    """
    preds_K = []
    T_prev = test_df['T01_TEMP'].iloc[0]
  
    for _ in range(pred_hrz):
        X = pd.DataFrame({'T01_TEMP': [T_prev]})
        T_next = ridge_model.predict(X)[0]
        preds_K.append(T_next)
        T_prev = T_next

    T_pred_F = (np.array(preds_K) - 273.15) * 9/5 + 32
    return T_pred_F


def autoregressive_predict_rf_univariate(rf_model, test_df, pred_hrz=24):
    """
    Univariate autoregressive forecast for Random Forest.
    Predicts T(t+1) using only T(t).
    
    Note: Model must have been trained using only 'T01_TEMP' as feature.
    
    Returns:
    - Temperature predictions in Fahrenheit
    """
    preds_K = []
    T_prev = test_df['T01_TEMP'].iloc[0]

    for _ in range(pred_hrz):
        X = pd.DataFrame({'T01_TEMP': [T_prev]})
        T_next = rf_model.predict(X)[0]
        preds_K.append(T_next)
        T_prev = T_next

    T_pred_F = (np.array(preds_K) - 273.15) * 9/5 + 32
    return T_pred_F

## Training and Testing Pipeline

In [ ]:
def training_and_testing_baselines(all_houses_reduced, optimization_results_R2C2_decent, 
                                   processed_houses_reduced, pred_hrz, duration, noise, 
                                   mode, batch_id):
    """
    Train and test R2C2, Ridge, and Random Forest models (both multivariate and univariate).
    """
    # Flatten and sort the house data
    new_house_data = {}
    total = 0
    house_data = OrderedDict(sorted(all_houses_reduced.items()))

    for house_group in house_data:
        sorted_dict = OrderedDict(sorted(house_data[house_group].items()))
        for house_id in sorted_dict:
            total += 1
            new_house_data[house_id] = total
            
    # Reorganize the dictionary to remove the sensor_count key
    flattened_results = {}
    for sensor_count, houses in optimization_results_R2C2_decent.items():
        for house_id, result in houses.items():
            flattened_results[house_id] = result

    flattened_data = {}
    for sensor_count, houses in processed_houses_reduced.items():
        for house_id, df in houses.items():
            flattened_data[house_id] = df

    # Process each house
    for house_id, house_data_df in flattened_data.items():
        rank = new_house_data[house_id]
        print(f"\nHouse ID {house_id} with rank {rank}")

        if house_id not in flattened_results:
            print(f"Skipping house {house_id} (no R2C2 results).")
            continue

        # Extract R2C2 parameters and initialize model
        results = flattened_results[house_id]
        optimal_params = results['optimal_params']

        Ri = optimal_params['Ri'][0]
        Re = optimal_params['Re'][0]
        Ci = optimal_params['Ci'][0]
        Ce = optimal_params['Ce'][0]
        Ai = optimal_params['Ai'][0]
        Ae = optimal_params['Ae'][0]
        roxP_hvac = optimal_params['roxP_hvac'][0]

        r2c2_model = R2C2Model(Ri, Re, Ci, Ce, Ai, Ae, roxP_hvac)

        # Train/test split
        test_size = 0.125
        num_test_samples = int(len(house_data_df) * test_size)
        split_index = len(house_data_df) - num_test_samples

        train_df = house_data_df.iloc[224:split_index].copy()
        test_df = house_data_df.iloc[split_index:].copy()
        print(f"Train shape: {train_df.shape}, Test shape: {test_df.shape}")
        
        # R2C2 Autoregressive predictions
        T_init = test_df['T01_TEMP'].iloc[0]
        T_ext_init = test_df['Text'].iloc[0]
        Te_init = (T_init + T_ext_init) / 2

        T_ext_seq = test_df['Text'].iloc[:pred_hrz].values
        u_seq = test_df['duty_cycle'].iloc[:pred_hrz].values
        ghi_seq = test_df['GHI'].iloc[:pred_hrz].values

        predictions_r2c2 = r2c2_model.autoregressive_predict(
            T_init, Te_init, T_ext_seq, u_seq, ghi_seq, dt=3600, pred_hrz=pred_hrz
        )

        # Ground truth in Fahrenheit
        ground_truth_K = test_df['T01_TEMP'].iloc[1:pred_hrz+1].values
        ground_truth_F = (ground_truth_K - 273.15) * 9/5 + 32

        rmse_r2c2 = np.sqrt(mean_squared_error(ground_truth_F, predictions_r2c2))
        print(f"[R2C2] RMSE = {rmse_r2c2:.2f} F")

        model_name = "R2C2"
        directory = f'/Users/ozanbaris/Documents/GitHub/TS-foundation-model/Aug18_ecobee_results_{noise}/{model_name}_new_ecobee_cov_{noise}_csv' 
        save_results_for_model(
            model_name=model_name,
            ground_truth=ground_truth_F,
            timestamp_forecast=test_df.index[:pred_hrz],
            forecast=predictions_r2c2,
            duration=duration,
            pred_hrz=pred_hrz,
            mode=mode,
            occupancy=rank,
            batch_id=batch_id,
            directory=directory
        )

        # Prepare training data for Ridge & RF (multivariate)
        X_train = train_df[['Text', 'duty_cycle', 'GHI', 'T01_TEMP']][:-1]
        y_train = train_df['T01_TEMP'][1:]
        print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
        
        # Fit and test Ridge (multivariate)
        ridge_model = Ridge()
        ridge_model.fit(X_train, y_train)

        predictions_ridge = autoregressive_predict_ridge(
            ridge_model, test_df, pred_hrz=pred_hrz
        )
        rmse_ridge = np.sqrt(mean_squared_error(ground_truth_F, predictions_ridge))
        print(f"[Ridge] RMSE = {rmse_ridge:.2f} F")
        
        model_name = "Ridge"
        directory = f'/Users/ozanbaris/Documents/GitHub/TS-foundation-model/Aug18_ecobee_results_{noise}/{model_name}_new_ecobee_cov_{noise}_csv' 
        save_results_for_model(
            model_name=model_name,
            ground_truth=ground_truth_F,
            timestamp_forecast=test_df.index[:pred_hrz],
            forecast=predictions_ridge,
            duration=duration,
            pred_hrz=pred_hrz,
            mode=mode,
            occupancy=rank,
            batch_id=batch_id,
            directory=directory
        )

        # Fit and test Random Forest (multivariate)
        rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
        rf_model.fit(X_train, y_train)

        predictions_rf = autoregressive_predict_rf(
            rf_model, test_df, pred_hrz=pred_hrz
        )
        rmse_rf = np.sqrt(mean_squared_error(ground_truth_F, predictions_rf))
        print(f"[RF] RMSE = {rmse_rf:.2f} F")
        
        model_name = "RandomForest"
        directory = f'/Users/ozanbaris/Documents/GitHub/TS-foundation-model/Aug18_ecobee_results_{noise}/{model_name}_new_ecobee_cov_{noise}_csv' 
        save_results_for_model(
            model_name=model_name,
            ground_truth=ground_truth_F,
            timestamp_forecast=test_df.index[:pred_hrz],
            forecast=predictions_rf,
            duration=duration,
            pred_hrz=pred_hrz,
            mode=mode,
            occupancy=rank,
            batch_id=batch_id,
            directory=directory
        )

        print(f"Finished House {rank}\n{'-'*40}")

        # Prepare univariate training data
        X_train_univ = train_df[['T01_TEMP']][:-1]
        print(f"X_train (Univariate) shape: {X_train_univ.shape}, y_train shape: {y_train.shape}")
        
        # Fit and test Ridge (univariate)
        ridge_model_univ = Ridge()
        ridge_model_univ.fit(X_train_univ, y_train)

        predictions_ridge_univ = autoregressive_predict_ridge_univariate(
            ridge_model_univ, test_df, pred_hrz=pred_hrz
        )
        rmse_ridge_univ = np.sqrt(mean_squared_error(ground_truth_F, predictions_ridge_univ))
        print(f"[Ridge_univ] RMSE = {rmse_ridge_univ:.2f} F")
        
        model_name = "Ridge_univ"
        directory = f'/Users/ozanbaris/Documents/GitHub/TS-foundation-model/Aug18_ecobee_results_{noise}/{model_name}_new_ecobee_cov_{noise}_csv' 
        save_results_for_model(
            model_name=model_name,
            ground_truth=ground_truth_F,
            timestamp_forecast=test_df.index[:pred_hrz],
            forecast=predictions_ridge_univ,
            duration=duration,
            pred_hrz=pred_hrz,
            mode=mode,
            occupancy=rank,
            batch_id=batch_id,
            directory=directory
        )

        # Fit and test Random Forest (univariate)
        rf_model_univ = RandomForestRegressor(n_estimators=100, random_state=42)
        rf_model_univ.fit(X_train_univ, y_train)

        predictions_rf_univ = autoregressive_predict_rf_univariate(
            rf_model_univ, test_df, pred_hrz=pred_hrz
        )
        rmse_rf_univ = np.sqrt(mean_squared_error(ground_truth_F, predictions_rf_univ))
        print(f"[RF_univ] RMSE = {rmse_rf_univ:.2f} F")

        model_name = "RF_univ"
        directory = f'/Users/ozanbaris/Documents/GitHub/TS-foundation-model/Aug18_ecobee_results_{noise}/{model_name}_new_ecobee_cov_{noise}_csv' 
        save_results_for_model(
            model_name=model_name,
            ground_truth=ground_truth_F,
            timestamp_forecast=test_df.index[:pred_hrz],
            forecast=predictions_rf_univ,
            duration=duration,
            pred_hrz=pred_hrz,
            mode=mode,
            occupancy=rank,
            batch_id=batch_id,
            directory=directory
        )

        print(f"Finished House {rank}\n{'-'*40}")

## Run Training Experiments Across Noise Levels

In [ ]:
batch_id = 0 
mode = None
duration = 448
pred_hrz = 64

noises = [0, 0.1, 0.2, 0.5, 1, 2, 5]
for noise in noises:
    dataset_name = f'house_data_csvs_{noise}'
    main_output_directory = f"/Users/ozanbaris/Documents/GitHub/TS-foundation-model/{dataset_name}"

    all_houses_reduced = read_csvs_to_dfs(main_output_directory)
    
    processed_houses_reduced = {}
    for sensor_count, houses in all_houses_reduced.items():
        processed_houses = {}
        for house_id, df in houses.items():
            if 'GHI' not in df.columns:
                print(f"Skipping house {house_id} due to missing 'GHI' column.")
                continue
            processed_houses[house_id] = process_house_data(df.copy())
        processed_houses_reduced[sensor_count] = processed_houses

    print("training and testing for NOISE: ", noise)
    training_and_testing_baselines(all_houses_reduced, optimization_results_R2C2_decent, 
                                  processed_houses_reduced, pred_hrz, duration, noise, 
                                  mode, batch_id)

## Results Analysis and Metrics Computation

In [ ]:
def process_results(directory, model_name, pred_hrz, duration, valid_occupancies):
    """
    Process and compute metrics for all occupancy values for a given model.

    Returns:
    - Dictionary with metrics averaged across occupancy values
    """
    all_metrics = []

    for occupancy in valid_occupancies:
        if noise == 0 and model_name not in ['R2C2', 'Ridge', 'RandomForest', 'Ridge_univ', 'RF_univ']:
            file_path = f"{directory}/{model_name}_new_ecobee_cov_csv/results_None_{occupancy}/{duration}_{pred_hrz}/{model_name}.csv"
        else:
            file_path = f"{directory}/{model_name}_new_ecobee_cov_{noise}_csv/results_None_{occupancy}/{duration}_{pred_hrz}/{model_name}.csv"
        
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            continue

        df = pd.read_csv(file_path)
        ground_truth = np.array([np.array(eval(gt)) for gt in df['GroundTruth']])
        forecast = np.array([np.array(eval(forecast)) for forecast in df['Forecast']])

        metrics = compute_metrics(ground_truth, forecast)
        all_metrics.append(metrics)

    # Average metrics across all occupancy values
    if all_metrics:
        avg_metrics = {
            'RMSE_all': np.mean([m['RMSE_all'] for m in all_metrics]),
            'RMSE_1st': np.mean([m['RMSE_1st'] for m in all_metrics if m['RMSE_1st'] is not None]),
            'RMSE_6th': np.mean([m['RMSE_6th'] for m in all_metrics if m['RMSE_6th'] is not None]),
            'RMSE_24th': np.mean([m['RMSE_24th'] for m in all_metrics if m['RMSE_24th'] is not None])
        }
    else:
        avg_metrics = None

    return avg_metrics


def analyze_noise_level(noise):
    """
    Analyze results for a specific noise level.
    """
    dataset_name = f'house_data_csvs_{noise}'
    new_house_data, flattened_results, flattened_data = prepare_experiment_data(
        dataset_name, optimization_results_R2C2_decent
    )
    
    directory = f'/Users/ozanbaris/Documents/GitHub/TS-foundation-model/Aug18_ecobee_results_{noise}'
    model_names = ['R2C2', 'Ridge', 'RandomForest', 'TimesFMCov', 'TimesFM', 'uni2ts', 
                   'uni2tsCov', 'chronos', 'LagLlama', 'moment', 'TimeGPT', 'TimeGPTCov', 
                   'RF_univ', 'Ridge_univ']
    pred_hrz = 64
    duration = 448
    
    valid_occupancies = extract_valid_occupancies(flattened_data, flattened_results, new_house_data)
    print(f"len of available house data {len(valid_occupancies)}")

    final_results = {}
    for model_name in model_names:
        print(f"Processing model: {model_name}")
        avg_metrics = process_results(directory, model_name, pred_hrz, duration, valid_occupancies)
        if avg_metrics:
            final_results[model_name] = avg_metrics

    # Save metrics summary
    metrics_df = pd.DataFrame.from_dict(final_results, orient='index')
    metrics_df.to_csv(f"{directory}/metrics_summary_{noise}.csv")
    print(metrics_df)
    
    return final_results


if __name__ == "__main__":
    for noise in [0, 0.1, 0.2, 0.5, 1]:
        analyze_noise_level(noise)

## LaTeX Table Generation (All Models)

In [ ]:
def generate_overleaf_table(noise_levels, final_results_per_noise, model_names):
    """
    Generates LaTeX table for all models across noise levels.
    """
    table_content = "\\begin{table*}[h!]\n"
    table_content += "    \\centering\n"
    table_content += "    \\caption{Results Across Different Noise Levels.}\n"
    table_content += "    \\label{tab:rmse}\n"
    table_content += "    \\resizebox{\\textwidth}{!}{%\n"
    table_content += "    \\begin{tabular}{l|" + "c" * len(model_names) + "}\n"
    table_content += "        \\toprule\n"
    table_content += "        \\textbf{Noise Level} & \\textbf{Metric} & " + " & ".join(f"\\textbf{{{name}}}" for name in model_names) + " \\\\\n"
    table_content += "        \\midrule\n"

    for noise, results in zip(noise_levels, final_results_per_noise):
        for metric in ['RMSE_all', 'RMSE_1st', 'RMSE_6th', 'RMSE_24th']:
            metric_values = {model: metrics.get(metric, "-") for model, metrics in results.items()}
            metric_df = pd.DataFrame.from_dict(metric_values, orient='index', columns=[metric])
            metric_df = metric_df.sort_values(metric, ascending=True)
            best_model = metric_df.index[0]
            second_best_model = metric_df.index[1]

            if metric == 'RMSE_all':
                row = f"        {noise} & RMSE\\textsubscript{{all}}"
            else:
                sub_metric = metric.split('_')[1]
                row = f"        & RMSE\\textsubscript{{{sub_metric}}}"

            for model in model_names:
                value = metric_values.get(model, "-")
                if value != "-":
                    value = f"{value:.2f}"
                    if model == best_model:
                        row += f" & \\textbf{{{value}}}"
                    elif model == second_best_model:
                        row += f" & \\underline{{{value}}}"
                    else:
                        row += f" & {value}"
                else:
                    row += " & -"
            row += " \\\\\n"
            table_content += row

    table_content += "        \\bottomrule\n"
    table_content += "    \\end{tabular}%\n"
    table_content += "    }\n"
    table_content += "\\end{table*}\n"

    return table_content


if __name__ == "__main__":
    noise_levels = [0, 0.1, 0.2, 0.5, 1]
    final_results_per_noise = []

    for noise in noise_levels:
        results = analyze_noise_level(noise)
        final_results_per_noise.append(results)
        
    model_names = ['R2C2', 'Ridge', 'RandomForest', 'TimesFMCov', 'uni2tsCov', 'TimeGPTCov', 
                   'TimesFM', 'uni2ts', 'TimeGPT', 'chronos', 'LagLlama', 'moment', 
                   'RF_univ', 'Ridge_univ']
    overleaf_table = generate_overleaf_table(noise_levels, final_results_per_noise, model_names)
    print(overleaf_table)

## LaTeX Table Generation (Univariate Models Only)

In [ ]:
def generate_overleaf_table_univariate(noise_levels, final_results_per_noise, model_names):
    """
    Generates LaTeX table for univariate models only.
    """
    table_content = "\\begin{table*}[h!]\n"
    table_content += "    \\centering\n"
    table_content += "    \\caption{Univariate Model Results Across Different Noise Levels.}\n"
    table_content += "    \\label{tab:rmse_univ}\n"
    table_content += "    \\resizebox{0.6\\textwidth}{!}{%\n"
    table_content += "    \\begin{tabular}{ll|" + "c" * len(model_names) + "}\n"
    table_content += "        \\toprule\n"
    table_content += "        \\textbf{Noise} & \\textbf{Metric} & " + " & ".join(f"\\textbf{{{name.replace('_', ' ')}}}" for name in model_names) + " \\\\\n"
    table_content += "        \\midrule\n"

    for i, (noise, results) in enumerate(zip(noise_levels, final_results_per_noise)):
        for j, metric in enumerate(['RMSE_all', 'RMSE_1st', 'RMSE_6th', 'RMSE_24th']):
            if not results:
                continue

            metric_values = {model: metrics.get(metric, "-") for model, metrics in results.items()}
            metric_df = pd.DataFrame.from_dict(metric_values, orient='index', columns=[metric])
            
            if not metric_df.empty:
                metric_df = metric_df.sort_values(metric, ascending=True)
                best_model = metric_df.index[0]
                second_best_model = metric_df.index[1] if len(metric_df.index) > 1 else None
            else:
                best_model, second_best_model = None, None

            noise_label = str(noise) if j == 0 else ""
            metric_label = f"RMSE\\textsubscript{{{metric.split('_')[1]}}}"
            row = f"        {noise_label} & {metric_label}"

            for model in model_names:
                value = metric_values.get(model, "-")
                if isinstance(value, (int, float)):
                    value_str = f"{value:.2f}"
                    if model == best_model:
                        row += f" & \\textbf{{{value_str}}}"
                    elif model == second_best_model:
                        row += f" & \\underline{{{value_str}}}"
                    else:
                        row += f" & {value_str}"
                else:
                    row += " & -"
            row += " \\\\\n"
            table_content += row
        
        if i < len(noise_levels) - 1:
            table_content += " \\midrule\n"

    table_content += "        \\bottomrule\n"
    table_content += "    \\end{tabular}%\n"
    table_content += "    }\n"
    table_content += "\\end{table*}\n"

    return table_content


if __name__ == "__main__":
    noise_levels = [0, 0.1, 0.2, 0.5, 1]
    final_results_per_noise = []

    for noise in noise_levels:
        print(f"\n--- Processing Noise Level: {noise} ---")
        dataset_name = f'house_data_csvs_{noise}'
        new_house_data, flattened_results, flattened_data = prepare_experiment_data(
            dataset_name, optimization_results_R2C2_decent
        )
        
        directory = f'/Users/ozanbaris/Documents/GitHub/TS-foundation-model/Aug18_ecobee_results_{noise}'
        model_names_univ = ['RF_univ', 'Ridge_univ']
        pred_hrz = 64
        duration = 448
        
        valid_occupancies = extract_valid_occupancies(flattened_data, flattened_results, new_house_data)
        print(f"len of available house data {len(valid_occupancies)}")

        final_results = {}
        for model_name in model_names_univ:
            print(f"Processing model: {model_name}")
            avg_metrics = process_results(directory, model_name, pred_hrz, duration, valid_occupancies)
            if avg_metrics:
                final_results[model_name] = avg_metrics

        final_results_per_noise.append(final_results)
    
    model_names_for_table = ['RF_univ', 'Ridge_univ']
    overleaf_table = generate_overleaf_table_univariate(noise_levels, final_results_per_noise, model_names_for_table)
    print("\n--- Generated LaTeX Table ---\n")
    print(overleaf_table)